# Init


### Install necessary requirements

In [ ]:
# Install necessary requirements
%pip install -r ../requirements.txt

### Change working directory and add jupyter reload

In [ ]:
# Change working directory to root
import os
if os.getcwd().endswith("notebooks"):
    %cd ..
    print(os.getcwd())

# Automatically reload changes in code
%load_ext autoreload
%autoreload 2

### Imports, logging and pandas configuration

In [ ]:
import logging
from typing import List
import pandas as pd
from association_finder.concept_drifts_finder import ConceptDriftsFinder
from association_finder.models import Transaction, ConceptDriftResult
from association_finder.concept_engineering import ConceptEngineering
from sklearn.model_selection import train_test_split
from association_finder.preprocessing import preprocess_dataset, split_X_y
from association_finder.one_vs_rest_classifier import OneVsRestClassifier, label_to_concept_transform_wrapper
from typing import Dict, Tuple, Optional
from dataclasses import dataclass
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Logs config
logging.basicConfig(level=logging.INFO)

# Pandas config
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

# Read, split and preprocess data

In [ ]:
np.random.seed(0)

# Read file
train_dataset_path = "datasets/big_mart_sales/Train-Set.csv"
df = pd.read_csv(train_dataset_path, index_col='ProductID')
target_column = "OutletSales"

# Drop rows with NaN values in the target column.
df.drop(df[df[target_column].isna()].index,inplace=True)

# Split
df_train, df_val = train_test_split(df, test_size=0.3)

# Preprocess    
df_train_prep, train_params = preprocess_dataset(df_train)

# Focusing on prominent columns:
good_columns = [column for column in ['Weight', 'FatContent', 'ProductVisibility', 'ProductType', 'MRP',
                'OutletID', 'EstablishmentYear', 'OutletSize', 'LocationType', 'OutletType', 'OutletSales']
                if column not in train_params.dropped_columns]
one_hot_columns = [column for column in ['FatContent','ProductType','OutletID','OutletSize','LocationType','OutletType']
                   if column not in train_params.dropped_columns]

In [ ]:
# Prepare data for training
X_train, y_train = split_X_y(df_train_prep, good_columns, train_params, one_hot_columns, target_column)
X_val, y_val = split_X_y(preprocess_dataset(df_val, train_params)[0], good_columns, train_params, one_hot_columns, target_column)

# Find rules and analyze them

In [ ]:
# Find association rules
concept_engineering = ConceptEngineering(min_confidence=0.1, min_support=0.1, diff_threshold=0.1, concept_drift_size_threshold=10)
concept_engineering.fit(X_train, df_train_prep[good_columns], target_column, one_hot_columns)
concept_engineering.concepts_df

# Build models

### Baseline model

In [ ]:
# Simple one vs rest classifier for baseline
one_vs_rest_classifier = OneVsRestClassifier()

In [ ]:
y_train_pred = one_vs_rest_classifier.fit_transform(X_train, y_train)
y_val_pred = one_vs_rest_classifier.transform(X_val)

print(f"Train accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"Validation accuracy: {accuracy_score(y_val, y_val_pred)}")

# Build model using rules

In [ ]:
# One vs rest classifier that uses rules (each label classifier uses its own rules)
label_to_transformation = {label: label_to_concept_transform_wrapper(concept_engineering, target_column, label) for label in y_train.unique()}
one_vs_rest_classifier = OneVsRestClassifier(label_to_transformation)

In [ ]:
y_train_pred = one_vs_rest_classifier.fit_transform(X_train, y_train)
y_val_pred = one_vs_rest_classifier.transform(X_val)

print(f"Train accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"Validation accuracy: {accuracy_score(y_val, y_val_pred)}")